In [ ]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [ ]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/3章'
# os.chdir(path)
# ! pip install japanize-matplotlib

# ３章　可視化の仕組みを構築する１０本ノック

In [1]:
import pandas as pd

# master data
m_store = pd.read_csv("../chapter01/input/m_store.csv")
m_area = pd.read_csv("../chapter01/input/m_area.csv")
# transacion data
order_data = pd.read_csv("../chapter01/input/tbl_order_202004.csv")

In [2]:
order_data = pd.merge(order_data, m_store, on="store_id", how="left")
order_data = pd.merge(order_data, m_area, on="area_cd", how="left")

In [3]:
order_data.loc[order_data["takeout_flag"] == 0, "takeout_name"] = "delivery"
order_data.loc[order_data["takeout_flag"] == 1, "takeout_name"] = "takeaway"
order_data.loc[order_data["status"] == 0, "status_name"] = "recept"
order_data.loc[order_data["status"] == 1, "status_name"] = "paid"
order_data.loc[order_data["status"] == 2, "status_name"] = "passed"
order_data.loc[order_data["status"] == 9, "status_name"] = "cancel"

In [4]:
order_data.head()

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name
0,79339111,49,C26387220,50,67393872,2020-04-01 11:00:00,2020-04-01 11:18:00,1,4144,1,浅草店,TK,東京,東京,takeaway,paid
1,18941733,85,C48773811,26,91834983,2020-04-01 11:00:00,2020-04-01 11:22:00,0,2877,2,目黒店,TK,東京,東京,delivery,passed
2,56217880,76,C66287421,36,64409634,2020-04-01 11:00:00,2020-04-01 11:15:00,0,2603,2,本郷店,TK,東京,東京,delivery,passed
3,28447783,190,C41156423,19,73032165,2020-04-01 11:00:00,2020-04-01 11:16:00,0,2732,2,栃木店,TO,北関東,栃木,delivery,passed
4,32576156,191,C54568117,71,23281182,2020-04-01 11:00:00,2020-04-01 11:53:00,0,2987,2,伊勢崎店,GU,北関東,群馬,delivery,passed


### ノック２１：店舗を絞り込んで可視化できるようにしてみよう

> `cf.`
>
> + []()

`observe()`

In [5]:
from IPython.display import display, clear_output
from ipywidgets import Dropdown


def order_by_store(val):
    '''
    この関数の処理はまず、
    　1. 結果エリアのクリア
       2. ドロップダウンの再描画
       3. 選択された店舗名に該当する注文データを抽出
       （ステータスコードが「paid(1)、passed(2)」を条件に付与。∵実際に売上として計上してよいデータに限定するため）
       4. 画面上に表示
    '''
    clear_output()
    display(dropdown)
    pick_data = order_data.loc[(order_data["store_name"] == val["new"]) & (order_data["status"].isin([1, 2]))]
    display(pick_data.head())

    
store_list = m_store["store_name"].tolist()
dropdown = Dropdown(options=store_list) #ドロップダウンオブジェクト生成
dropdown.observe(order_by_store, names="value") #ドロップダウンオブジェクト生成変化を検知したら関数実行
display(dropdown)

Dropdown(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛西店', '…

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib


def graph_by_store(val):
    '''
    この関数の処理はまず、
    　1~3. order_by_storeと同様
       4. 描画用に一時的なデータフレームをコピー
       5. 日付型に再定義（object型→datetime型）
       6. 日付型のカラムをインデックス化（∵日時等でリサンプリング(再集計)を行うため）
       7. 「D」日を意味し、日単位でリサンプリングを行った結果を描画
    '''
    clear_output()
    display(dropdown2)
    pick_data = order_data.loc[(order_data["store_name"] == val["new"]) & (order_data["status"].isin([1, 2]))]
    temp = pick_data[["order_accept_date", "total_amount"]].copy()
    temp.loc[:, "order_accept_date"] = pd.to_datetime(temp["order_accept_date"])
    temp.set_index("order_accept_date", inplace=True)
    temp.resample("D").sum().plot()

    
dropdown2 = Dropdown(options=store_list)
dropdown2.observe(graph_by_store, names="value")
display(dropdown2)

Dropdown(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛西店', '…

##### - reverse engineering

In [ ]:
pick_data = order_data.loc[(order_data["store_name"] == "あきる野店") & (order_data["status"].isin([1, 2]))] 
pick_data.head()

In [ ]:
temp = pick_data[["order_accept_date", "total_amount"]].copy()
temp.iloc[:, 0]= pd.to_datetime(temp["order_accept_date"])
temp.dtypes

In [ ]:
temp.set_index("order_accept_date").head()

### ノック２２：複数店舗の詳細を可視化できるようにしてみよう

In [12]:
from ipywidgets import SelectMultiple

def order_by_multi(val):
    '''
    この関数の処理
    1. 結果エリアのクリア
    2. セレクトエリアの再描画
    3. .isin(val["new"])関数を用いて、選択された複数を取得
    4. 画面上に表示
    '''
    clear_output()
    display(select)
    pick_data = order_data.loc[(order_data["store_name"].isin(val["new"])) & (order_data["status"].isin([1, 2]))]
    display(pick_data.head())

    
# 複数の店舗を選ぶときは、ShiftキーやCt rlキーを押しながらクリック
select = SelectMultiple(options=store_list) #マルチセレクトオブジェクト生成
select.observe(order_by_multi, names="value")
display(select)

SelectMultiple(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛…

In [13]:
def graph_by_multi(val):
    clear_output()
    display(select2)
    fig = plt.figure(figsize=(17, 4))
    plt.subplots_adjust(wspace=0.25, hspace=0.6)
    
    i = 0
    
    for trg in val["new"]:
        pick_data = order_data.loc[(order_data["store_name"] == trg)&(order_data["status"].isin([1, 2]))]
        temp = order_data[["order_accept_date", "total_amount", "store_name"]].copy()
        temp.loc[:, "order_accept_date"] = pd.to_datetime(temp["order_accept_date"])
        temp.set_index("order_accept_date", inplace=True)
        i += 1
        ax = fig.add_subplot(1, len(val["new"]), i)
        ax.plot(temp.resample("D").sum())
        ax.set_title(trg)
        

select2 = SelectMultiple(options=store_list)
select2.observe(graph_by_multi, names="value")
display(select2)

SelectMultiple(options=('昭島店', 'あきる野店', '足立店', '北千住店', '綾瀬店', '荒川店', '東尾久店', '板橋店', '高島平店', '稲城店', '江戸川店', '西葛…

### ノック２３：スライドバーを用いてオーダー件数を調べてみよう

### ノック２４：トグルボタンで地域データを抽出しよう

### ノック２５：日付を指定してデータを抽出してみよう

### ノック２６：ストーリーを考えてデータを構築しよう

### ノック２７：キャンセルの理由を分析してみよう

### ノック２８：仮説を検証してみよう

### ノック２９：ストーリーをもとにパーツやデータを組み合わせてダッシュボードを作ろう

### ノック３０：ダッシュボードを改善しよう